In [ ]:
!pip list | grep spark

In [ ]:
import findspark

findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test").getOrCreate()
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

# Create a SparkConf object
conf = SparkConf().setAppName("HBaseIntegration")

# Add the HBase connector JAR
conf.set("spark.jars.packages", "org.apache.hbase.connectors.spark:hbase-spark:2.4.8")

# Create a Spark session with the HBase configurations
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Set HBase connection properties
hbase_conf = {
    "spark.hadoop.hbase.zookeeper.quorum": "localhost",  # Change this to your HBase Zookeeper address
    "spark.hadoop.hbase.zookeeper.property.clientPort": "2181",  # Default Zookeeper client port
    "spark.hadoop.hbase.master": "localhost:16000",  # HBase master host and port
}

# Set the configuration for the session
for key, value in hbase_conf.items():
    spark.conf.set(key, value)

# Query from HBase Table (e.g., `my_table`)
df = (
    spark.read.format("org.apache.hadoop.hbase.spark")
    .option("hbase.table", "my_table")
    .option("hbase.columns.mapping", "rowkey STRING :key, cf:name STRING")
    .load()
)

# Show the result
df.show()